<a href="https://colab.research.google.com/github/Tlen33/Data-parsing/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек

%matplotlib inline
import re
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fixed_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Базы/hh.csv') # Путь, куда скачали файл базы headhunter
fixed_df.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,0,"Мужчина , 42 года , родился 6 октября 1976",27 000 руб.,Старший администратор,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц\n\nСтарший администр...,Трц Ривьера,Старший администратор по сохраности активов,Среднее специальное образование 1994 Спту 26,28.04.2019 12:17,Имеется собственный автомобиль
1,1,"Мужчина , 41 год , родился 14 января 1978",60 000 руб.,Старший IT инженер,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца\n\nСтарший IT инже...,"Эскорт-Центр, Москва, ЗАО",Старший инженер сервисного центра,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...,28.04.2019 11:44,Не указано
2,2,"Мужчина , 44 года , родился 21 февраля 1975",65 000 руб.,"Системный администратор, Системный инженер","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев\n\nСистемный адми...,"СПССПК ""ЭКОПТИЦА""",Системный администратор,Высшее образование 2003 Московский государств...,28.04.2019 10:36,Не указано
3,3,"Мужчина , 43 года , родился 9 февраля 1976",70 000 руб.,"Web-программист, Web-администратор, Web-мастер","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы 5 лет 9 месяцев\n\nWeb-программист...,"ГБУ ""МДОО""",специалист отдела интернет-проектов,Высшее образование 2002 Российский государств...,28.04.2019 10:29,Не указано
4,4,"Мужчина , 39 лет , родился 14 июня 1979",45 000 руб.,Старший системный администратор \ начальник от...,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев\n\nСтарший систем...,"ООО ""СервисСофт""",Старший системный администратор,Высшее образование 2007 Российский государств...,28.04.2019 10:15,Не указано


In [ ]:
# Данные о поле и возрасте
def getParameterSexAge(arg):
  out = 0
  #Если М, то 1. По умолчанию 0 - Ж
  if "Муж" in arg:
    sex = 'М'
  elif "Жен" in arg:
    sex = 'Ж'
  else:
    sex = "Не указан"
  # текущий год - год рождения
  year_tec = 2020
  if (len(arg) > 7):
    out = year_tec - int(re.findall(r'\d{4}', arg)[0])

  return sex, str(out)

# Зарплата
def getParameterSalary(arg):
    global salaryStr, num #т.к. используем, меняем в функции
    # Сначала получаем чистое число, убираем лишние знаки
    salaryStr = 0
    arg=str(arg)  
    salaryStr = int(re.sub(r'[^0-9]+', '', arg))
    curr_ = re.sub('[0-9]','',arg).replace('.','')

    # Конвертируем в рубли, если валюта
    if 'USD' in curr_ :   salaryStr = float(salaryStr)*75
    elif 'KZT' in curr_:  salaryStr = float(salaryStr)*0.17
    elif 'грн' in curr_:  salaryStr = float(salaryStr)*2.6
    num =salaryStr 
    return salaryStr

# Данные о городе
def getParameterCity(arg):
  millionCities = "Новосибирск Екатеринбург Нижний Новгород Казань Челябинск Омск Самара Ростов-на-Дону Уфа Красноярск Пермь Воронеж Волгоград Сочи"
  sarg=arg.split(',')
  item = sarg[0] # Берем только город
  item= item.strip() # Удаляем пробелы
  if item == "Москва":          return "Москва"
  if item == "Санкт-Петербург": return "Санкт-Петербург"
  if item in millionCities:     return item
  else: return "малый или за пределами России город"

# Данные о перееезде
def getParameterMove(arg):
  sarg=arg.split(',')
  item2 = sarg[1] # Берем только город
  item2= item2.strip() # Удаляем пробелы
  if item2 == "не готов к переезду": return "Не готов к переезду"
  if item2 == '':     return 'Не указано'
  else: return "Готов к переезду"

# Данные о командировках
def getParameterMissions(arg):
  sarg=arg.split(',')
  item3 = sarg[2] # Берем только город
  item3= item3.strip() # Удаляем пробелы
  if item3 == "не готов к командировкам": return "Не готов к командировкам"
  if item3 == '':     return 'Не указано'
  else: return "Готов к командировкам"

# Данные о желаемой занятости
def getParameterEmployment(arg):
  result = ''
  if ("стажировка" in arg)|("volunteer" in arg)|("work placement" in arg): result += 'стажировка '
  if ("частичная занятость" in arg)|("part" in arg):  result += 'частичная занятость '
  if ("проектная работа" in arg)|("project" in arg):  result += 'проектная работа '
  if ("полная занятость" in arg)|("full" in arg):     result += 'полная занятость'
  if result=='':
    result='Не указано'
  return result

# Данные о желаемом графике работы
def getParameterSchedule(arg):
  result = ''
  if ("гибкий график" in arg)|("flexible" in arg):    result += 'гибкий график '
  if ("полный день" in arg)|("full" in arg):     result += 'полный день '
  if ("сменный график" in arg)|("shift" in arg):   result += 'сменный график '
  if ("удаленная работа" in arg)|("remote" in arg): result += 'удаленная работа'
  if result=='':
    result='Не указано'
  return result

# Данные об образовании
def getParameterEducation(arg):
  result = ''
  arg = arg.lower()
  if ("высшее" in arg)|("higher education"  in arg): result += 'высшее '
  if ("среднее специальное"  in arg) | ("special"  in arg) | ("колледж"  in arg) | ("college"  in arg) : result += 'среднее специальное '
  if ("неоконченное высшее"  in arg) | ("incomplete"  in arg): result += 'неоконченное высшее '
  if ("среднее образование"  in arg) | ("secondary education"  in arg): result += 'среднее образование'
  if result=='':
    result='Не указано'
  return result

# Данные об опыте работы
def getParameterExperience(arg):
  sarg=arg.split(" ", 5)
  years = 0
  months = 0
  if len(sarg)!=6: sarg.append(0);sarg.append('');sarg.append(0);sarg.append('')
  try:
    #print(sarg)
    years = int(sarg[2])
  except ValueError:
    years = 0

  try:
    months = int(sarg[4])
  except ValueError:
    months = 0
  return 12*years+months

In [ ]:
df = fixed_df.drop(fixed_df.columns[[0,3,8,9,11,12]],axis=1 ) # удаляем неинтересующие нас столбцы

In [ ]:
df.head()

,"Пол, возраст",ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"Мужчина , 42 года , родился 6 октября 1976",27 000 руб.,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц\n\nСтарший администр...,Среднее специальное образование 1994 Спту 26
1,"Мужчина , 41 год , родился 14 января 1978",60 000 руб.,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца\n\nСтарший IT инже...,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...
2,"Мужчина , 44 года , родился 21 февраля 1975",65 000 руб.,"Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев\n\nСистемный адми...,Высшее образование 2003 Московский государств...
3,"Мужчина , 43 года , родился 9 февраля 1976",70 000 руб.,"Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы 5 лет 9 месяцев\n\nWeb-программист...,Высшее образование 2002 Российский государств...
4,"Мужчина , 39 лет , родился 14 июня 1979",45 000 руб.,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев\n\nСтарший систем...,Высшее образование 2007 Российский государств...


In [ ]:
df.insert(3,'Переезд', np.array([df['Город'][i] for i in range(len(df))]))      # Создаем отдельную колонку под Переезд 
df.insert(4,'Командировки', np.array([df['Город'][i] for i in range(len(df))])) # Создаем отдельную колонку под Командировки

In [ ]:
df.head()

,"Пол, возраст",ЗП,Город,Переезд,Командировки,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"Мужчина , 42 года , родился 6 октября 1976",27 000 руб.,"Липецк , не готов к переезду , не готов к кома...","Липецк , не готов к переезду , не готов к кома...","Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц\n\nСтарший администр...,Среднее специальное образование 1994 Спту 26
1,"Мужчина , 41 год , родился 14 января 1978",60 000 руб.,"Сочи , не готов к переезду , готов к командиро...","Сочи , не готов к переезду , готов к командиро...","Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца\n\nСтарший IT инже...,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...
2,"Мужчина , 44 года , родился 21 февраля 1975",65 000 руб.,"Липецк , готов к переезду , готов к командировкам","Липецк , готов к переезду , готов к командировкам","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев\n\nСистемный адми...,Высшее образование 2003 Московский государств...
3,"Мужчина , 43 года , родился 9 февраля 1976",70 000 руб.,"Москва , м. Марьино , не готов к переезду , г...","Москва , м. Марьино , не готов к переезду , г...","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы 5 лет 9 месяцев\n\nWeb-программист...,Высшее образование 2002 Российский государств...
4,"Мужчина , 39 лет , родился 14 июня 1979",45 000 руб.,"Тула , не готов к переезду , готов к редким ко...","Тула , не готов к переезду , готов к редким ко...","Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев\n\nСтарший систем...,Высшее образование 2007 Российский государств...


In [ ]:
# Создаем список в которых лежат объекты наших функций
funcList = [getParameterSexAge,getParameterSalary, getParameterCity,getParameterMove,getParameterMissions,getParameterEmployment,getParameterSchedule,getParameterExperience,getParameterEducation]

# Проходимся по каждой колонке и применяем к каждой из колонок соответствующую функцию
for i in range(len(df.columns)):
  df[df.columns[i]] = [funcList[i](cell) for cell in df[df.columns[i]]]

In [ ]:
df.head()

,"Пол, возраст",ЗП,Город,Переезд,Командировки,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"(М, 44)",27000.0,малый или за пределами России город,Не готов к переезду,Не готов к командировкам,полная занятость,полный день,73,среднее специальное
1,"(М, 42)",60000.0,Сочи,Не готов к переезду,Готов к командировкам,частичная занятость полная занятость,гибкий график полный день,230,высшее
2,"(М, 45)",65000.0,малый или за пределами России город,Готов к переезду,Готов к командировкам,полная занятость,полный день удаленная работа,237,высшее
3,"(М, 44)",70000.0,Москва,Готов к переезду,Готов к командировкам,полная занятость,гибкий график полный день удаленная работа,69,высшее
4,"(М, 41)",45000.0,малый или за пределами России город,Не готов к переезду,Готов к командировкам,частичная занятость полная занятость,полный день сменный график,161,высшее


In [ ]:
df.insert(0,'Пол', np.array([df['Пол, возраст'][i][0] for i in range(len(df))]))          # Создаем отдельную колонку под Пол 
df.insert(1,'Возраст', np.array([int(df['Пол, возраст'][i][1]) for i in range(len(df))])) # Создаем отдельную колонку под Возраст
df.drop('Пол, возраст', axis = 1, inplace = True)                                         # Убираем лишнюю колонку

In [ ]:
df.head()

,Пол,Возраст,ЗП,Город,Переезд,Командировки,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,М,44,27000.0,малый или за пределами России город,Не готов к переезду,Не готов к командировкам,полная занятость,полный день,73,среднее специальное
1,М,42,60000.0,Сочи,Не готов к переезду,Готов к командировкам,частичная занятость полная занятость,гибкий график полный день,230,высшее
2,М,45,65000.0,малый или за пределами России город,Готов к переезду,Готов к командировкам,полная занятость,полный день удаленная работа,237,высшее
3,М,44,70000.0,Москва,Готов к переезду,Готов к командировкам,полная занятость,гибкий график полный день удаленная работа,69,высшее
4,М,41,45000.0,малый или за пределами России город,Не готов к переезду,Готов к командировкам,частичная занятость полная занятость,полный день сменный график,161,высшее
